In [43]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [44]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [45]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,10,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [46]:
# getting rid of non-ascii characters from the reviews as it breaks csv

df_recent.review_content = df_recent.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_new.review_content = df_new.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_fave.review_content = df_fave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_rant.review_content = df_rant.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df_anom.review_content = df_anom.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [47]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [48]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



43525
39132


In [49]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
760,Cascade Primordial Noir,1.9,-1.9,Cascade Brewing,"Tap (Sori Taproom, Not Tallinn) (5:4:3:1:6=1.9...",FatPhil
761,AC Golden Colorado Native Amber Lager,5.0,+2.2,Coors Brewing Company (Molson Coors),So many beers are full of hops these days. Its...,BarrelLees49
762,Rhinegeist Lynx,1.3,-2,Rhinegeist Brewery,"Hahah, it said lunx on the beer menu! Slight o...",cheap
763,Clifford Porter,0.5,-3.1,Clifford Brewing Co.,This is not a porter. It tastes like an IPA. V...,fredthecat


In [50]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [51]:
from selenium import webdriver
import time

In [52]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [53]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 247 new beers to search for!


In [54]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver1')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [55]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Elland Lucifer\n🇬🇧 Strong Ale - English • 5.7%...
1,Derby Pale Mild Moment\n🇬🇧 Bitter - Ordinary /...
2,Beartown Polar Eclipse\n🇬🇧 Stout - Oatmeal • 4...
3,"Connecticut Valley Wind, Sands, and Stars\n🇺🇸 ..."
4,Great Divide Heyday Modern IPA\n🇺🇸 IPA • 5.8%\...
...,...
242,TRVE / Brewery Bhavana Despair is a Siren\n🇺🇸 ...
243,Deschutes Fresh Funk Wild IPA\n🇺🇸 IPA - Sour /...
244,Resident We are All In This Together\n🇺🇸 IPA -...
245,Frecon Estate Apfelwein\n🇺🇸 Apple Cider • 9.0%...


In [56]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [57]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [58]:
# need to add in add check here for price


In [59]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
10,À La Fût + L'Hermite | Triple du Nouveau Monde...,À La Fût + L'Hermite | Triple du Nouveau Monde...,🇨🇦,Tripel,NaN,-,-,3.80,1
89,Four Sons Hopprentice\n🇺🇸 ISA - Session IPA0\n...,Four Sons Hopprentice,🇺🇸,ISA - Session IPA,0.0,-,-,3.35,4
96,À La Dérive Boathouse 4\n🇨🇦 IPA - Hazy / NEIPA...,À La Dérive Boathouse 4,🇨🇦,IPA - Hazy / NEIPA,NaN,-,-,4.07,3
187,Cycle Barrel Aged Barleywine and Stout Blended...,Cycle Barrel Aged Barleywine and Stout Blended...,🇺🇸,Stout - Imperial,NaN,-,-,4.30,1
196,Prairie Vinyl\n🇺🇸 Stout - Imperial\n-\n-\n4.10...,Prairie Vinyl,🇺🇸,Stout - Imperial,NaN,-,-,4.10,1
231,Crooked Stave Colorado Reserva Goldbar Apricot...,Crooked Stave Colorado Reserva Goldbar Apricot,🇺🇸,Sour / Wild Beer,0.0,91,77,3.79,14


In [60]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [61]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
89,Four Sons Hopprentice\n🇺🇸 ISA - Session IPA0\n...,🇺🇸,ISA - Session IPA,0.0,-,-,3.35,4,Four Sons Hopprentice
96,À La Dérive Boathouse 4\n🇨🇦 IPA - Hazy / NEIPA...,🇨🇦,IPA - Hazy / NEIPA,NaN,-,-,4.07,3,À La Dérive Boathouse 4
187,Cycle Barrel Aged Barleywine and Stout Blended...,🇺🇸,Stout - Imperial,NaN,-,-,4.30,1,Cycle Barrel Aged Barleywine and Stout Blended...
196,Prairie Vinyl\n🇺🇸 Stout - Imperial\n-\n-\n4.10...,🇺🇸,Stout - Imperial,NaN,-,-,4.10,1,Prairie Vinyl


In [62]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 3 new beers that were not found or returned errors. They will be added to the no search list.


In [63]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)